In [1]:
from langchain.prompts import PromptTemplate
from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnableSequence

In [2]:
import os
from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv())
openai_api_key = os.environ["OPENAI_API_KEY"]

In [3]:
prompt = PromptTemplate.from_template(
    "You are a helpful assistant. Write a short poem about {topic}."
)

In [4]:
from langchain_openai import ChatOpenAI 
# ✅ Step 2: Initialize an LLM (OpenAI GPT)
llm = ChatOpenAI(model="gpt-3.5-turbo", temperature=0.7)

In [5]:
parser = StrOutputParser()

In [6]:
chain = RunnableSequence(first=prompt, middle=[llm], last=parser)

In [7]:
result = chain.invoke({"topic": "space exploration"})
print("\n[RunnableSequence Output]\n", result)



[RunnableSequence Output]
 In the vast expanse beyond our skies,
Adventurers boldly reach new highs.
Through galaxies and nebulae they roam,
Exploring the great unknown.

They navigate the cosmic sea,
Seeking out what lies unseen.
From distant planets to shining stars,
They journey on to lands afar.

With courage and curiosity as their guide,
They push the boundaries far and wide.
In search of answers to questions profound,
They explore the mysteries that abound.

So let us cheer for those who dare,
To venture out into the cosmic air.
For in their quest to understand,
They pave the way for future explorers to expand.


In [8]:
from langchain_core.runnables import RunnablePassthrough
chain=RunnablePassthrough()

## RunnablePassthrough
* It does not do anything to the input data.
* Let's see it in a very simple example: a chain with just RunnablePassthrough() will output the original input without any modification.

In [9]:
from langchain_core.runnables import RunnablePassthrough
chain=RunnablePassthrough()

In [10]:
chain.invoke("Hello, world!")

'Hello, world!'

In [11]:
from langchain_core.runnables import RunnableSequence, RunnablePassthrough

passthrough1 = RunnablePassthrough()
passthrough2 = RunnablePassthrough()

chain = RunnableSequence(first=passthrough1, last=passthrough2)

print(chain.invoke("Hello world"))

Hello world


## RunnableLambda
* To use a custom function inside a LCEL chain we need to wrap it up with RunnableLambda.
* Let's define a very simple function to create Russian lastnames:## RunnableLambda
* To use a custom function inside a LCEL chain we need to wrap it up with RunnableLambda.
* Let's define a very simple function to create Russian lastnames:

In [12]:
from langchain_core.runnables import RunnableLambda

In [13]:
def russian_lastname(name: str) -> str:
    return f"{name}ovich"

In [14]:
chain = RunnablePassthrough() | RunnableLambda(russian_lastname)

In [16]:
chain.invoke("Vikas ")

'Vikas ovich'

In [17]:
from langchain_core.runnables import RunnableLambda, RunnableMap

stripper = RunnableLambda(lambda x: x.strip())
lowercase = RunnableLambda(lambda x: x.lower())
length = RunnableLambda(lambda x: len(x))


parallel = RunnableMap({
    "cleaned": stripper,
    "lowered": lowercase,
    "length": length
})

In [18]:
result = parallel.invoke("   LangChain Rocks!   ")
print(result)

{'cleaned': 'LangChain Rocks!', 'lowered': '   langchain rocks!   ', 'length': 22}


## RunnableParallel
* We will use RunnableParallel() for running tasks in parallel.
* This is probably the most important and most useful Runnable from LangChain.
* In the following chain, RunnableParallel is going to run these two tasks in parallel:
    * operation_a will use RunnablePassthrough.
    * operation_b will use RunnableLambda with the russian_lastname function.

In [19]:
from langchain_core.runnables import RunnableParallel

In [20]:
chain = RunnableParallel(
    {
        "operation_a": RunnablePassthrough(),
        "operation_b": RunnableLambda(russian_lastname)
    }
)

In [21]:
chain.invoke("prince")

{'operation_a': 'prince', 'operation_b': 'princeovich'}

In [22]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate

In [23]:
prompt = ChatPromptTemplate.from_template("tell me a curious fact about {soccer_player}")

output_parser = StrOutputParser()


In [24]:
def russian_lastname_from_dictionary(person):
    return person["name"] + "ovich"

In [25]:
chain = RunnableParallel(
    {
        "operation_a": RunnablePassthrough(),
        "soccer_player": RunnableLambda(russian_lastname_from_dictionary),
        "operation_c": RunnablePassthrough(),
    }
) | prompt | llm | output_parser

In [26]:
chain.invoke({"soccer_player": "Cristiano Ronaldo", "name": "Cristiano Ronaldo"})

'There is no official player named "Cristiano Ronaldoovich." Cristiano Ronaldo, whose full name is Cristiano Ronaldo dos Santos Aveiro, is a Portuguese professional footballer widely regarded as one of the greatest players of all time.'

In [27]:
chain.invoke({"name": "prince", "soccer_player": "messi"})

'Princeovich is not a real surname, but rather a title commonly used in European monarchies to indicate a prince of a particular region or country.'

# RunnableBranch


In [28]:
from langchain_core.runnables import RunnableBranch, RunnableLambda


math_handler = RunnableLambda(lambda x: "Calling calculator tool...")
text_handler = RunnableLambda(lambda x: "Running text summarizer...")
default_handler = RunnableLambda(lambda x: f"No match, fallback: {x}")


router = RunnableBranch(
    (lambda x: "math" in x.lower(), math_handler),
    (lambda x: "text" in x.lower(), text_handler),
    default_handler
)


result = router.invoke("math")
print(result)  # Output: Calling calculator tool...
result = router.invoke("text")
print(result)  # Output: Running text summarizer...

Calling calculator tool...
Running text summarizer...


In [29]:
from langchain.prompts import PromptTemplate
from langchain_openai import ChatOpenAI
from langchain_core.runnables import RunnableSequence

In [30]:
qa_prompt = PromptTemplate.from_template("Answer the question: {query}")
qa_chain = qa_prompt | ChatOpenAI()

In [31]:

joke_prompt = PromptTemplate.from_template("Tell a joke about {query}")
joke_chain = joke_prompt | ChatOpenAI()


router = RunnableBranch(
    (lambda x: "joke" in x["query"], joke_chain),
    qa_chain  # default: answer question
)


result = router.invoke({"query": "joke about AI"})
print(result)

content="Why did the AI break up with his girlfriend? Because he couldn't handle her emotional bandwidth!" additional_kwargs={'refusal': None} response_metadata={'token_usage': {'completion_tokens': 19, 'prompt_tokens': 14, 'total_tokens': 33, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'id': 'chatcmpl-Bc44aTv0cdnNqRWvpE12SmMUNHMM2', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None} id='run--1b16ee7a-2b7f-4706-a3b0-6f686b8c1b7a-0' usage_metadata={'input_tokens': 14, 'output_tokens': 19, 'total_tokens': 33, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}}


In [32]:
print("The End")

The End
